<a href="https://colab.research.google.com/github/ishimwe-jean-claude/aiclass/blob/main/full_code_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain-openai -q
!pip install langchain-community -q
!pip install langchain-experimental -q

In [ ]:
from google.colab import userdata
from langchain_openai import OpenAIEmbeddings
class ZhipuAI_embeddings:
    def __init__(self, model_name: str = 'embeddings-3'):
        self.model_name = model_name
        self.base_url = "https://open.bigmodel.cn/api/paas/v4"
        self.embedding = self._init_model()
    def _init_model(self) -> OpenAIEmbeddings:
        return OpenAIEmbeddings(
            model=self.model_name,
            base_url=self.base_url,
            api_key=userdata.get("ishimwe_k")
        )
embeddings =ZhipuAI_embeddings().embedding

In [ ]:
from langchain_openai import ChatOpenAI
client  = ChatOpenAI(
    base_url ="https://open.bigmodel.cn/api/paas/v4/",
    api_key = userdata.get("ishimwe_k"),
    model = "glm-4.5"
)

In [ ]:
client.invoke("hello").cotent

AIMessage(content='Hello! How can I assist you today? 😊', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 8, 'total_tokens': 21, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'glm-4.5', 'system_fingerprint': None, 'id': '20250912174909ce8b9ff152de445e', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--409ade20-de86-4a5b-b226-600d75eb6e16-0', usage_metadata={'input_tokens': 8, 'output_tokens': 13, 'total_tokens': 21, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}})

In [ ]:
from langchain_core.documents import Document
from langchain_experimental.text_splitter import SemanticChunker
from langchain_community.document_loaders.text import TextLoader
from langchain_community.document_loaders import PyPDFLoader
def doc_parsing(file_path) -> list[Document]:
    if file_path.endswith("pdf"):
        doc = PyPDFLoader(file_path=file_path)
        doc = doc.load()
        semantic_splitter = SemanticChunker(
            embeddings=embeddings,
            breakpoint_threshold_type="percentile",
            breakpoint_threshold_amount=95
        )
        full_text = doc[0].page_content if doc else ""
        if not full_text:
            return []
        raw_chunks = semantic_splitter.split_text(full_text)
        print(f" the number of chucks :{len(raw_chunks)}")
        docs = [Document(page_content=chunk, metadata=doc[0].metadata) for chunk in raw_chunks]
        return docs
    elif file_path.endswith(".txt"):
        doc = TextLoader(file_path=file_path)
        doc =doc.load()
        semantic_splitter = SemanticChunker(
            embeddings=embeddings,
            breakpoint_threshold_type="percentile",
            breakpoint_threshold_amount=95
        )
        full_text = doc[0].page_content if doc else ""
        if not full_text:
            return []
        raw_chunks = semantic_splitter.split_text(full_text)
        docs = [Document(page_content=chunk, metadata=doc[0].metadata) for chunk in raw_chunks]
        return docs
    else:
       return []

In [ ]:
from langchain_core.documents import Document
from langchain_experimental.text_splitter import SemanticChunker
from langchain_community.document_loaders.text import TextLoader
from langchain_community.document_loaders import PyPDFLoader
def doc_parsing(file_path) -> list[Document]:
    if file_path.endswith("pdf"):
        doc = PyPDFLoader(file_path=file_path)
        doc = doc.load()
        semantic_splitter = SemanticChunker(
            embeddings=embeddings,
            breakpoint_threshold_type="percentile",
            breakpoint_threshold_amount=95
        )
        full_text = doc[0].page_content if doc else ""
        if not full_text:
            return []
        raw_chunks = semantic_splitter.split_text(full_text)
        print(f" the number of chucks :{len(raw_chunks)}")
        docs = [Document(page_content=chunk, metadata=doc[0].metadata) for chunk in raw_chunks]
        return docs
    elif file_path.endswith(".txt"):
        doc = TextLoader(file_path=file_path)
        doc =doc.load()
        semantic_splitter = SemanticChunker(
            embeddings=embeddings,
            breakpoint_threshold_type="percentile",
            breakpoint_threshold_amount=95
        )
        full_text = doc[0].page_content if doc else ""
        if not full_text:
            return []
        raw_chunks = semantic_splitter.split_text(full_text)
        docs = [Document(page_content=chunk, metadata=doc[0].metadata) for chunk in raw_chunks]
        return docs
    else:
       return []

In [ ]:
s_template = """
Your name is AICLASS assistant a smart, yet sassy assistant working at takenolab, your have a better knowledge of takenolab operations,
your task is to be supportive, provide proper guidance to students that are having troubles with the course content,
you have to answer them direct and precise, if you dont have any advice to them dont generate any respose kindly
tell them so.
When you receive:
• question: the student’s problem
• context: optionally, the content of any uploaded documents

If `context` is non-empty, you **must** use it to inform your answer. If you don’t have enough information
from the question and context combined, tell the student you can’t help further.

student problem:
{question}

uploaded document context (if any):
{context}

your smart advice or solution:

"""
new_template = """
Your name is AICLASS assistant, a smart, yet sassy assistant working at takenolab. You have a deep knowledge of takenolab operations.
Your task is to be supportive, provide proper guidance to students who are having trouble with the course content.
You must answer them directly and precisely. If you don't have any advice for them, kindly tell them so and do not generate any other response.

When you receive:
• chat_history: the previous turns of the conversation (if any)
• question: the student’s current problem
• context: optionally, the content of any uploaded documents relevant to the current question

If `context` is non-empty, you **must** use it to inform your answer. If you don’t have enough information
from the question and context combined, tell the student you can’t help further.
If `chat_history` is provided, use it to understand the full context of the current question.

<chat_history>
{chat_history}
</chat_history>

student problem:
{question}

uploaded document context (if any):
{context}

your smart advice or solution:

"""
SUB_QUERY_TEMPLATE = """
You are a helpful assistant that generates multiple search queries based on a single input query.
Generate {num_queries} diverse search queries related to the user's question, which can be used to retrieve relevant documents.
The queries should be concise and cover different aspects or angles of the original question.

Original Question: {question}

Generated Queries:
-

"""

In [ ]:
from langchain_core.documents import Document
from langchain_experimental.text_splitter import SemanticChunker
from langchain_community.document_loaders.text import TextLoader
from langchain_community.document_loaders import PyPDFLoader
def doc_parsing(file_path) -> list[Document]:
    if file_path.endswith("pdf"):
        doc = PyPDFLoader(file_path=file_path)
        doc = doc.load()
        semantic_splitter = SemanticChunker(
            embeddings=embeddings,
            breakpoint_threshold_type="percentile",
            breakpoint_threshold_amount=95
        )
        full_text = doc[0].page_content if doc else ""
        if not full_text:
            return []
        raw_chunks = semantic_splitter.split_text(full_text)
        print(f" the number of chucks :{len(raw_chunks)}")
        docs = [Document(page_content=chunk, metadata=doc[0].metadata) for chunk in raw_chunks]
        return docs
    elif file_path.endswith(".txt"):
        doc = TextLoader(file_path=file_path)
        doc =doc.load()
        semantic_splitter = SemanticChunker(
            embeddings=embeddings,
            breakpoint_threshold_type="percentile",
            breakpoint_threshold_amount=95
        )
        full_text = doc[0].page_content if doc else ""
        if not full_text:
            return []
        raw_chunks = semantic_splitter.split_text(full_text)
        docs = [Document(page_content=chunk, metadata=doc[0].metadata) for chunk in raw_chunks]
        return docs
    else:
       return []

In [ ]:
s_template = """
Your name is AICLASS assistant a smart, yet sassy assistant working at takenolab, your have a better knowledge of takenolab operations,
your task is to be supportive, provide proper guidance to students that are having troubles with the course content,
you have to answer them direct and precise, if you dont have any advice to them dont generate any respose kindly
tell them so.
When you receive:
• question: the student’s problem
• context: optionally, the content of any uploaded documents

If `context` is non-empty, you **must** use it to inform your answer. If you don’t have enough information
from the question and context combined, tell the student you can’t help further.

student problem:
{question}

uploaded document context (if any):
{context}

your smart advice or solution:

"""
new_template = """
Your name is AICLASS assistant, a smart, yet sassy assistant working at takenolab. You have a deep knowledge of takenolab operations.
Your task is to be supportive, provide proper guidance to students who are having trouble with the course content.
You must answer them directly and precisely. If you don't have any advice for them, kindly tell them so and do not generate any other response.

When you receive:
• chat_history: the previous turns of the conversation (if any)
• question: the student’s current problem
• context: optionally, the content of any uploaded documents relevant to the current question

If `context` is non-empty, you **must** use it to inform your answer. If you don’t have enough information
from the question and context combined, tell the student you can’t help further.
If `chat_history` is provided, use it to understand the full context of the current question.

<chat_history>
{chat_history}
</chat_history>

student problem:
{question}

uploaded document context (if any):
{context}

your smart advice or solution:

"""
SUB_QUERY_TEMPLATE = """
You are a helpful assistant that generates multiple search queries based on a single input query.
Generate {num_queries} diverse search queries related to the user's question, which can be used to retrieve relevant documents.
The queries should be concise and cover different aspects or angles of the original question.

Original Question: {question}

Generated Queries:
-

"""

In [ ]:
from langchain_core.tools import tool

@tool
def define_term(term:str):
  """  Returns a one-sentence definition of a given AI term
     (e.g., “Transformer”, “Embedding”)."""
  return f"The definition of {term} is ..."

@tool
def summarize_notes(text: str):
  """ Takes student notes (a paragraph) and returns a
  concise summary (2–3 sentences). """
  return f"The summary of the notes is ..."

from langchain_core.prompts import PromptTemplate

model = client.bind_tools([define_term, summarize_notes])
prompt_template = PromptTemplate.from_template(template=s_template)
new_template = prompt_template | model


response = define_term.invoke("natural language processing")
print(response)


In [ ]:

from langchain_core.prompts import PromptTemplate
import gradio as gr
from langchain.chains import LLMChain
from langchain_core.documents import Document
from typing import TypedDict, List
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_core.messages import AIMessage, HumanMessage, BaseMessage
from typing import Optional,Tuple, Dict

vector_store = InMemoryVectorStore(embeddings)
def call_assistant(question:str="", context_docs:List[Optional[Document]]=None,chat_history: List[Tuple[str, str]]=None):
    prompt_template = PromptTemplate.from_template(template=new_template)
    chain = prompt_template | client
    history_str = ""
    if chat_history:
        for human_msg, ai_msg in chat_history:
            history_str += f"User: {human_msg}\n Assistant: {ai_msg}\n"
    context_text =""
    if context_docs:
         context_text = "\n\n".join(d.page_content for d in context_docs)
    response = chain.invoke({
        "question": question,
        "context": context_text,
        "chat_history": history_str
    })
    return response.content
def generate_sub_queries(original_question: str, num_queries: int = 3) -> List[str]:
    sub_query_prompt = PromptTemplate.from_template(template=SUB_QUERY_TEMPLATE)
    sub_query_chain = sub_query_prompt | client.client

    response = sub_query_chain.invoke({
        "question": original_question,
        "num_queries": num_queries
    })

    queries = [q.strip() for q in response.content.split('-') if q.strip()]
    print(f"Generated sub-queries: {queries}")
    return queries
def retrieve_and_answer_with_history(question: str, chat_history: List[Dict])->str:
    # retrieved = vector_store.similarity_search(question)
    formatted_chat_history_for_llm = []
    for msg in chat_history:
        if msg["role"] == "user":
            current_user_msg = msg["content"]
        elif msg["role"] == "assistant":
            formatted_chat_history_for_llm.append((current_user_msg, msg["content"]))
            current_user_msg = None
    if len(vector_store.store.items()) >= 0:
        transformed_queries = generate_sub_queries(question, num_queries=3)
        all_retrieved_docs = []
        seen_doc_contents = set()
        for query in transformed_queries:
            retrieved_for_query = vector_store.similarity_search(query)
            for doc in retrieved_for_query:
                if doc.page_content not in seen_doc_contents:
                    all_retrieved_docs.append(doc)
                    seen_doc_contents.add(doc.page_content)
        ai_response = call_assistant(question, context_docs=all_retrieved_docs, chat_history=formatted_chat_history_for_llm)
        return ai_response
    ai_response = call_assistant(question,chat_history=formatted_chat_history_for_llm)
    return ai_response
def doc_loader(file_path):
    docs = doc_parsing(file_path)
    if not docs:
        return "No content found or processed in the document."
    _ = vector_store.add_documents(documents=docs)
    return docs[0].page_content[:200] + "..."

def interface():
    iface = gr.ChatInterface(
        fn=retrieve_and_answer_with_history,
        chatbot=gr.Chatbot(height=200, type='messages', label="Assistant"),
        textbox=gr.Textbox(lines=2,submit_btn=True ),
        title="Takenolab AIClass Assistant (Conversational RAG)",
        type='messages',
        description="Ask a question about your course content and get smart advice, supporting multi-turn conversations.",
    )
    docs_interface = gr.Interface(
        fn=doc_loader,
        inputs=gr.File(label="Choose a file to upload",
                       type='filepath',
                       file_count='single',
                       show_label=True
                       ),
        description="Upload a document to run retrieval‐augmented generation.",
        outputs=gr.TextArea()
    )
    table = gr.TabbedInterface(
        [iface,docs_interface],
        tab_names= ['Chat', "Upload File for RAG"],
        title="LLM, RAG AND PROMPTS, Text Generation"
    )
    table.launch(debug=True, server_port=3000)


In [ ]:
interface()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://7c065424015ee43b7a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.


KeyboardInterrupt: 